##### Script to produce CCA S2S forecast and assess associated skill, using NCEP subseasonal rainfall  (4 inits per month), vs IMD data.

Authors: Á.G. Muñoz (agmunoz@iri.columbia.edu), Andrew W. Robertson and Remi Cousin

This version: 25 Jun 2018, Modified by AWR

First version: 12 Dec 2017; First IPython version (Jupyter Notebook), 24 Jun 2018 by AWR

### Output:
* Several skill maps for assessment of deterministic forecast, in the output folder.
* CPT scripts used to assess skill, in the scripts folder.
* Downloaded input files, in the input folder.

### Notes:
1. Old data in the input folder is deleted at the beginning of the process!
2. *Weekly* initializations available per month are used, concatenated.
3. The T coordinate has been faked, so CPT can deal with all the initializations.
4. Rainfall observations are IMD data (India) at 0.25 deg. (Land only)
5. IMD Lead week DEFINITIONS: week 1 3-9; week 2 10-16; week 3+4 17-30:
6. Training: Uses the calendar month containing the forecast start day (not optimal!)
7. Grads output  
8. climo is defined as the mean of 3 perturbed runs
9. REAL TIME FORECAST USES the 4 6-hourly forecasts from 'fday' as well as 24Z; NB: fday will be YESTERDAY's date in real time, if run in the morning NY time
    

In [3]:
####START OF USER-MODIFIABLE SECTION######

# Forecast date
mon='Jun' 	# Forecast month 
fyr=2018 	# Forecast year
fday=20 	# Forecast day  (Yesterday in real time)

# Forecast lead interval
wk=2 		# week lead label
day1=10 	# First daily lead selected. For NCEP model, L1=1 is accum tp after 1 day
day2=16	 	# Last daily lead selected

nday=day2-day1+1	# Length of target period (days)
  
# Spatial domain for predictor
nla1=32 	# Northernmost latitude
sla1=12 	# Southernmost latitude
wlo1=74 	# Westernmost longitude
elo1=92 	# Easternmost longitude
# Spatial domain for predictand
nla2=27 	# Northernmost latitude
sla2=22 	# Southernmost latitude
wlo2=80 	# Westernmost longitude
elo2=89 	# Easternmost longitude

# Working directory
workdir = '/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2'

# PATH to CPT root directory
cptdir='/Users/andy/Dropbox/pgm/stats/CPT/CPT15/15.7.3/'

# S2S Database key

# AWR key
key='f7a5c7a57103d4ed1be1224f814b01b77951a6c20a4cbe85dd6b1942af7e479b11e3443a819d6bd601ac35070c7e5667e84e23f5'

####END OF USER-MODIFIABLE SECTION######

In [13]:
# Prepare folders (OS commands require IPython)
print("Creating working folders, if not already there...")
%cd $workdir
!mkdir -p input
!mkdir -p output
!mkdir -p scripts
!rm -Rf input/model_*.tsv input/obs_*.tsv  #comment if deletion of old input files is not desired.
!rm -Rf scripts/*

%cd input
# Set up some parameters
import os
os.system('export CPT_BIN_DIR='+cptdir)
#! export CPT_BIN_DIR=${cptdir}

Creating working folders, if not already there...
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2
Parent directory:
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/input


0

In [14]:
print("Downloading hindcasts/observations/forecasts ...")
! pwd
  
# Download hindcasts (NCEP)

url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/add/4./div/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+mon+')/VALUES/S/7/STEP/dup/S/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.S/replaceGRID/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/c://name//water_density/def/998/%28kg/m3%29/:c/div//mm/unitconvert//name/(tp)/def/grid://name/%28T%29/def//units/%28months%20since%201960-01-01%29/def//standard_name/%28time%29/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/:grid/use_as_grid/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
print("\n Hindcasts URL: \n\n "+url)
os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > model_precip_"+mon+".tsv")
#! curl -g -k -b '__dlauth_id='$key'' ''$url'' > model_precip_${mo}.tsv

/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/input

 Hindcasts URL: 

 http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/3./mul/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/add/4./div/X/74/92/RANGE/Y/12/32/RANGE/L1/9/16/VALUES/%5BL1%5Ddifferences/S/(Jun)/VALUES/S/7/STEP/dup/S/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.S/replaceGRID/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/c://name//water_density/def/998/%28kg/m3%29/:c/div//mm/unitconvert//name/(tp)/def/grid://name/%28T%29/def//units/%28months%20since%201960-01-01%29/def//standard_name/%28time%29/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/:grid/use_as_grid/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv


0

In [15]:
# Download IMD observations

url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/S/('+mon+')/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/'+str(nday)+'/runningAverage/'+str(nday)+'/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv'
print("\n Obs data URL: \n\n "+url)
os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > obs_precip_"+mon+".tsv")
#curl -g -k -b '__dlauth_id='$key'' ''$url'' > obs_precip_${mo}.tsv


 Obs data URL: 

 http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.control/.sfc_precip/.tp/S/(0000%201%20Jan%201999)/(0000%2031%20Dec%202010)/RANGEEDGES/L1/9/16/VALUES/%5BL1%5Ddifferences/S/(Jun)/VALUES/S/7/STEP/L1/S/add/0/RECHUNK//name//T/def/2/%7Bexch%5BL1/S%5D//I/nchunk/NewIntegerGRID/replaceGRIDstream%7Drepeat/use_as_grid/SOURCES/.IMD/.NCC1-2005/.v4p0/.rf/T/(days%20since%201960-01-01)/streamgridunitconvert/T/(1%20Jan%201999)/(31%20Dec%202011)/RANGEEDGES/T/7/runningAverage/7/mul/T/2/index/.T/SAMPLE/nip/dup/T/npts//I/exch/NewIntegerGRID/replaceGRID/dup/I/5/splitstreamgrid/%5BI2%5Daverage/sub/I/3/-1/roll/.T/replaceGRID/-999/setmissing_value/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/16/Jan/1901/ensotime/12./16/Jan/1960/ensotime/%3Agrid/use_as_grid/%5BX/Y%5D%5BT%5Dcptv10.tsv


0

In [16]:
# Download forecast file

url='http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/X/70/100/RANGE/Y/0/40/RANGE/L1/'+str(day1-1)+'/'+str(day2)+'/VALUES/%5BL1%5Ddifferences/L1/removeGRID/S/(0000%20'+str(fday)+'%20'+mon+')/VALUES/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div//mm/unitconvert/SOURCES/.NOAA/.NCEP/.EMC/.CFSv2/.6_hourly_rotating/.FLXF/.surface/.PRATE/S/(0000%20'+str(fday)+'%20'+mon+'%20'+str(fyr)+')/(0000%20'+str(fday+1)+'%20'+mon+'%20'+str(fyr)+')/RANGE/%5BM%5Daverage/%5BL%5D1/0.0/boxAverage/%5BX/Y%5DregridAverage/L/'+str(day1)+'/'+str(day2)+'/RANGEEDGES/%5BL%5Daverage/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div/(mm/day)/unitconvert/'+str(nday)+'/mul//units/(mm)/def/exch/sub/X/'+str(wlo1)+'/'+str(elo1)+'/RANGE/Y/'+str(sla1)+'/'+str(nla1)+'/RANGE/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/1/Jan/2001/ensotime/12.0/1/Jan/2001/ensotime/%3Agrid/addGRID/T//pointwidth/0/def/pop//name/(tp)/def//units/(mm)/def//long_name/(precipitation_amount)/def/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv'
print("\n Forecast URL: \n\n "+url)
os.system("curl -g -k -b '__dlauth_id="+key+"' '"+url+"' > modelfcst_precip_fday"+str(fday)+".tsv")
!curl -g -k -b '__dlauth_id='$key'' ''$url'' > modelfcst_precip_fday${fday}.tsv


 Forecast URL: 

 http://iridl.ldeo.columbia.edu/SOURCES/.ECMWF/.S2S/.NCEP/.reforecast/.perturbed/.sfc_precip/.tp/%5BM%5Daverage/X/70/100/RANGE/Y/0/40/RANGE/L1/9/16/VALUES/%5BL1%5Ddifferences/L1/removeGRID/S/(0000%2020%20Jun)/VALUES/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div//mm/unitconvert/SOURCES/.NOAA/.NCEP/.EMC/.CFSv2/.6_hourly_rotating/.FLXF/.surface/.PRATE/S/(0000%2020%20Jun%202018)/(0000%2021%20Jun%202018)/RANGE/%5BM%5Daverage/%5BL%5D1/0.0/boxAverage/%5BX/Y%5DregridAverage/L/10/16/RANGEEDGES/%5BL%5Daverage/%5BS%5Daverage/c%3A//name//water_density/def/998/(kg/m3)/%3Ac/div/(mm/day)/unitconvert/7/mul//units/(mm)/def/exch/sub/X/74/92/RANGE/Y/12/32/RANGE/grid%3A//name/(T)/def//units/(months%20since%201960-01-01)/def//standard_name/(time)/def//pointwidth/1/def/1/Jan/2001/ensotime/12.0/1/Jan/2001/ensotime/%3Agrid/addGRID/T//pointwidth/0/def/pop//name/(tp)/def//units/(mm)/def//long_name/(precipitation_amount)/def/-999/setmissing_value/%5BX/Y%5D%5BT%5Dcptv10.tsv
/

In [22]:
# Create CPT script
% cd ../scripts
!pwd
! rm -r params
print("---------------------------------------------------")
print("Producing CPT script")

#cat  <<< '#!/bin/bash 
#'${cptdir}'CPT.x <<- END

# Opens CCA
%store 611 > params  
# Opens X input file
%store 1 >> params
file='../input/model_precip_'+mon+'.tsv'
%store file >> params
# Nothernmost latitude
%store nla1 >> params
# Southernmost latitude
%store sla1 >> params
# Westernmost longitude
%store wlo1 >> params
# Easternmost longitude
%store elo1 >> params

# Minimum number of modes
%store 1 >> params
# Maximum number of modes
%store 10 >> params
# Opens forecast (X) file
%store 3 >> params
file='../input/modelfcst_precip_fday'+str(fday)+'.tsv'
%store file >> params

# Opens Y input file
%store 2 >> params
file='../input/obs_precip_'+mon+'.tsv'
%store file >> params
# Northernmost latitude
%store nla2 >> params
# Southernmost latitude
%store sla2 >> params
# Westernmost longitude
%store wlo2 >> params
# Easternmost longitude
%store elo2 >> params

# Minimum number of modes
%store 1 >> params
# Maximum number of modes
%store 10 >> params
# Minimum number of CCA modes
%store 1 >> params
# Maximum number of CCAmodes
%store 5 >> params

# X training period
%store 4 >> params
# First year of X training period
%store 1901 >> params
# Y training period
%store 5 >> params
# First year of Y training period
%store 1901 >> params

# Goodness index
%store 531 >> params
# Kendalls tau
%store 3 >> params

# Option: Length of training period
%store 7 >> params
# Length of training period 
%store 55 >> params
# Option: Length of cross-validation window
%store 8 >> params
# Enter length
%store 3 >> params

# Turn ON Transform predictand data
%store 541 >> params
# Turn ON zero bound for Y data
#%store 542 >> params
# Turn ON synchronous predictors
%store 545 >> params
# Turn ON p-values for skill maps
#%store 561 >> params

# Missing value options
%store 544 >> params
# Missing value X flag:
blurb='-999'
%store blurb >> params
# Maximum % of missing values
%store 10 >> params
# Maximum % of missing gridpoints
%store 10 >> params
# Number of near-neighbors
%store 1 >> params
# Missing value replacement : best-near-neighbors
%store 4 >> params
# Y missing value flag
blurb='-999'
%store blurb >> params
# Maximum % of missing values
%store 10 >> params
# Maximum % of missing stations
%store 10 >> params
# Number of near-neighbors
%store 1 >> params
# Best near neighbor
%store 4 >> params

#554 # Transformation seetings
#1   #Empirical distribution

#######BUILD MODEL AND VALIDATE IT  !!!!!

# select output format
%store 131 >> params
# GrADS format
%store 3 >> params

# Cross-validation
%store 311 >> params

# save goodness index
%store 112 >> params
file='../output/PRCP_Kendallstau_CCA_'+mon+'_wk'+str(wk)
%store file >> params

# cross-validated skill maps
%store 413 >> params
# save Spearmans Correlation
%store 2 >> params
file='../output/PRCP_Spearman_CCA_'+mon+'_wk'+str(wk)
%store file >> params

# cross-validated skill maps
%store 413 >> params
# save 2AFC score
%store 3 >> params
file='../output/PRCP_2AFC_CCA_'+mon+'_wk'+str(wk)
%store file >> params

# cross-validated skill maps
%store 413 >> params
# save 2AFC score
%store 10 >> params
file='../output/PRCP_RocBelow_CCA_'+mon+'_wk'+str(wk)
%store file >> params

# cross-validated skill maps
%store 413 >> params
# save 2AFC score
%store 11 >> params
file='../output/PRCP_RocAbove_CCA_'+mon+'_wk'+str(wk)
%store file >> params

#######FORECAST(S)  !!!!!
# Probabilistic (3 categories) maps
%store 455 >> params
# Output results
%store 111 >> params
# Forecast probabilities
%store 501 >> params
file='../output/PRCP_CCAFCST_PROB_'+mon+'_wk'+str(wk)
%store file >> params
#502 # Forecast odds

#Exit submenu
%store 0 >> params

#0 # Stop saving  (not needed in newest version of CPT)

# Exit
%store 0 >> params


/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/scripts
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/scripts
---------------------------------------------------
Producing CPT script
Writing '611' (int) to file 'params'.
Writing '1' (int) to file 'params'.
Writing 'file' (str) to file 'params'.
Writing 'nla1' (int) to file 'params'.
Writing 'sla1' (int) to file 'params'.
Writing 'wlo1' (int) to file 'params'.
Writing 'elo1' (int) to file 'params'.
Writing '1' (int) to file 'params'.
Writing '10' (int) to file 'params'.
Writing '3' (int) to file 'params'.
Writing 'file' (str) to file 'params'.
Writing '2' (int) to file 'params'.
Writing 'file' (str) to file 'params'.
Writing 'nla2' (int) to file 'params'.
Writing 'sla2' (int) to file 'params'.
Writing 'wlo2' (int) to file 'params'.
Writing 'elo2' (int) to file 'params'.
Writing '1' (int) to file 'params'.
Writing '10' (int) to file 'params'.
Writing '1' (int) to file 'params'.
Writing '5' (int) to file 'params'.
Writing '4' 

In [35]:
print('Executing CPT ...')
#%cd /Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/scripts
os.system(cptdir+'CPT.x < params > CPT_stout_'+mon+'_wk'+str(wk))
% cd ..

print('Done!')

Executing CPT ...
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/scripts
/Users/andy/Dropbox/pgm/angelpgm/Jun2018_RealTime/v2/scripts
Done!
